# List of all available endpoints

#### The notebook initialization

In [1]:
from IPython.display import HTML
from init import *

## Endpoint: /

In [2]:
HTML(requests.get(server).text)

## Endpoint: /version

In [3]:
requests.get(f'{server}/version').json().split('\n')

['mc-0.1.1 built=[Oct 10 2022 12:45:43] #deps=2',
 '  KernelModelArgs-0.5.0 #deps=0',
 '  Model-0.5.0 #deps=1',
 '    Parameter-0.2.0 #deps=0',
 '']

# Prometheus metrics

In [4]:
requests.get(f'{server}/metrics').text.split('\n')

['']

# Model

In [5]:
model = {'TimeStart': 0,
 'TimeSteps': 1000,
 'NumPaths': 100000,
 'updaters': [
     {'name': 'IndependentGaussian'},
     {'name': 'BrownianMotion', 'start': 0.1, 'args': [0.2,2]}
 ],
 'evaluations': [{'state': 0, 'time': 2}]
}
model_json = json.dumps(model)
model_json

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentGaussian"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2}]}'

In [6]:
requests.post(f'{server}/model',model_json).json()

{'histograms': [],
 'mean': [0.4940077066421509],
 'names': ['BrownianMotion'],
 'npaths': [100000],
 'run_time_seconds': {'mean': 0.16691353917121887,
  'n': 1024,
  'skewness': None,
  'stddev': 0.00018337005167268217},
 'skewness': [-0.01316062081605196],
 'stddev': [2.834304094314575],
 'time_points': [2.0],
 'time_steps': [1000]}

## Endpoint: "/functions"

In [7]:
functions = requests.get(f'{server}/functions').json()
functions

[{'args': ['level', 'valueOnCrossing', 'direction', 'actionOnCrossing'],
  'desc': 'Barrier (up/down) with windows',
  'desc_md': "\n# Barrier\n\nA barrier state starts from some initial value S0.\nOn each computation step we verify the barrier crossing condition, and if it is satisfied, the barrier state will\nbe set to STATE=valueOnCrossing.\n\nFor direction 'up':\n\n    if(Sold<level and Snew>=level)\n        barrierState = valueOnCrossing;\n\nFor direction 'down':\n\n    if(Sold>level and Snew<=level)\n        barrierState = valueOnCrossing;\n\n## References\n\nThe barrier function expectes exactly one reference: an **underlying**.\n\n### [ref 0]\n\nThe underlying process.\n\n## Arguments\n\n### [arg 0] level\nBarrier level\n\n### [arg 1] valueOnCrossing\nThe value we set when the barrier is crossed.\n\n### [arg 2] direction\n\nThis is the barrier crossing condition.\n\n| value | direction  |\n| ------------- |-------------:|\n| -1 | down          |\n|  0 | any (up or down) |\n| +1

In [8]:
from IPython.display import Markdown as md
doc = ''
for f in functions:
    doc += f['desc_md'] + '\n\n'
md(doc)


# Barrier

A barrier state starts from some initial value S0.
On each computation step we verify the barrier crossing condition, and if it is satisfied, the barrier state will
be set to STATE=valueOnCrossing.

For direction 'up':

    if(Sold<level and Snew>=level)
        barrierState = valueOnCrossing;

For direction 'down':

    if(Sold>level and Snew<=level)
        barrierState = valueOnCrossing;

## References

The barrier function expectes exactly one reference: an **underlying**.

### [ref 0]

The underlying process.

## Arguments

### [arg 0] level
Barrier level

### [arg 1] valueOnCrossing
The value we set when the barrier is crossed.

### [arg 2] direction

This is the barrier crossing condition.

| value | direction  |
| ------------- |-------------:|
| -1 | down          |
|  0 | any (up or down) |
| +1 | up |

### [arg 3] action
If the barrier crossing condition is satisfied, then the new barrier
state will be changed as:

| value | action | the new barrier state will be ... |
| ------------- |-------------:|------|
| 0 | set | ... set to **valueOnCrossing** |
| 1 | increment | ... incremented by **valueOnCrossing** |
| 2 | multiply | ... multiplied by **valueOnCrossing** |

### [arg 4+] Number of time windows

The number of time windows (say **m**) when the barrier will be checked.
The next **m * 2** parameters will be treated as [from,to) pairs, time windows
where the barrier condition will be checked. If **m==0**, the barrier condition
will be checked on all time step.

## Examples

### S0=0, args=[1000,1,-1,0,0]

The barrier inital state is 0. We check on any time step (because arg[4]==0)
the the barrier level==1000 will be crossed in the down direction (for example,
from 1000.5 to 1000). If the condition is NOT fulfilled, we don't change the
barrier state. Otherwise, we set the barrier value (arg[3]==0 means 'set')
to value 1 (because arg[1]==1). Obvioulsy, only the first crossing over
the barrier will have an effect, when we change the barrier state from 0
(because S0==0) to 1 (because arg[1]==1 and arg[3]==0)

### S0=0, args=[1000,1,-1,0,2,111.1,222.2,333.3,444.4]

The same as above, but wee only check the barrier condition if the time
is in one of the two (because arg[5]==2) windows:

- [111.1,222.2)
- [333.3,444.4)

### S0=0, args=[1000,1,-1,1,2,111.1,222.2,333.3,444.4]

The same as above, we 'count' (arg[3]==1 is the increment)
how many times the barrier is crossed in the 'down' direction.



# BrownianMotion
The function expects either 2 arguments (drift,diffusion) OR 2 references to (drift,diffusion) processes.
    


Use ref1 as a source, and saves into ref2 a correlated
normally distributed random variable. The correlation parameter
"rho" is taken from "args".





# Division
Division of the two processes.

If abs(denominator)<=eps, than the division result will be not-a-number.



# BrownianMotion
The function expects either 2 arguments (drift,diffusion) OR 2 references to (drift,diffusion) processes.
    




# Linear1DInterpolation

Perform interpolation between [Xmin,Xmax]. The number of interpolation points
is deduced from the number of arguments passed to the function.
The first two items are Xmin,Xmax, another arguments are Y-values on the
regular grtid between [Xmin,Xmax]. Thus, the total nummber of arguments
must be at least 4.

### Example: args=[1,2,3,4,5]
there are three interpolation points (5-2=3):

- Xmin = 1.0
- Xmax = 2.0
- Y(X=1.0) = 3
- Y(X=1.5) = 4
- Y(X=2.0) = 5

The interpolation function argument is passed as 'Xref'. If Xref=-1, then the argument is time.
Otherwise, it is a state number to which will be used as an 'x'-argument.

Linear1DInterpolation returns Y(Xmin) if X<Xmin and Y(Xmax) if X>Xmax.

Linear1DInterpolation does not use the old state value.
Thus, the starting process state value is ignored.




# Multiplication



# Polynom

The polynom order is taken from the number of passed arguments.
Minimum number is 1, which will be treated as constant function.
Otherwise it will compute a0+a1*x+a2*x*x+..., where 'x'-is the
polynom argumnt (see below) and a0,a1... are arguments of the
function.

The polynopm argument 'x' is passed as 'Xref'. If Xref=-1, then the argument is time.
Otherwise, it is a state number to which will be used as an 'x'-argument.









